# urls_to_video.py

In [ ]:
import os
import pandas as pd
from pytube import YouTube

def make_dir(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

def link_save(link):
    yt = YouTube(link)
    video_name = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').first().download()
    re_name = os.path.join(video_dir, link.split('v=')[-1]+'.mp4')
    os.rename(video_name, re_name)
    return re_name

In [ ]:
## if __name__ == "__main__"
video_dir = os.path.join('.','data','video')
make_dir(video_dir)

df = pd.read_csv(os.path.join('.', 'data', 'links.csv')) ######변경######
save_file = os.path.join('.', 'data', 'links_videos.txt')
count = 0

with open(save_file, 'a') as f:
    f.write('links,videos\n')

for link in set(df.links):
    try: # 접근불가 영상
        name = link_save(link)
        with open(save_file, 'a') as f:
            f.write('{},{}\n'.format(link, name))
        count += 1
        print(count)
    except:
        print('Except')

# video_to_text.py

# video_to_text_outloop_proto.ipynb

In [ ]:
import os
import pandas as pd
import shutil
# import save_text ##바꿔야 함

In [ ]:
def save_text(video): #다운로드에 있는 파일을 경로변경 및 해당 파일의 경로 return
    video = video.split("\\")[-1].split(".")[0]
    source = "C:/Users/eunsun/Downloads/" + video + ".txt" ################변경
    target =  "./data/text/" + video + ".txt"
    shutil.move(source, target)
    return target

In [ ]:
def make_dir(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

In [ ]:
text_dir = os.path.join('.','data','text')
make_dir(text_dir)

In [ ]:
df = pd.read_csv(os.path.join('.', 'data', 'links_videos.txt'))

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Voyagerx 를 통한 자막 추출  
# 
# ###   환경 :
#     1) 윈도우  + python 3.7.x  
#     2) 현재 디렉토리에 Chrome_webdriver 존재
#     3) 현재 디렉토리 중 하위 폴더 (movie)에 동영상 파일만 존재  
#     4) 다운로드는 디폴트로 설정된 곳으로 바로 다운로드 됨  
#     


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm_notebook
import time
import pywinauto
import os


save_file = os.path.join('.','data','links_videos_texts.txt')
with open(save_file, 'a') as f:
    f.write('links,videos,texts\n')


# 브라우저 실행 
browser = webdriver.Chrome("C:/Users/eunsun/tobigs/chromedriver.exe") #############본인경로에 맞게 변경
browser.get('https://vrew.voyagerx.com/ko/try/')
wait = WebDriverWait(browser, 4)
element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'close')))
browser.find_element_by_class_name('close').click()
time.sleep(2)
print('-'*50)

# 크롤링 루프
for idx, row in df.iterrows():
    video = row['videos'][2:]
    print(video,'추출')
    
    # 기본 변수 생성
    path = os.getcwd()+'\\'+video ################변경
    check = browser.find_elements_by_class_name('word')[:10]
    new_check = browser.find_elements_by_class_name('word')[:10]
    # 파일 탭으로 이동
    browser.find_element_by_tag_name('button').click()
    
    # 업로드 클릭
    wait = WebDriverWait(browser, 5)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]')))
    browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]').click()

    # 업로드 실행
    time.sleep(1)
    app = pywinauto.application.Application().connect(title_re='열기')
    app.열기.Edit.SetText(path)
    time.sleep(1)
    app.열기.Button.click()
    try :
        app.열기.Button.click()
    except:
        pass

    # 변환 확인
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'blue-button')))
    browser.find_element_by_class_name('blue-button').click()
    new_check = browser.find_elements_by_class_name('word')[:10]

    i=0
    while check == new_check:
        new_check = browser.find_elements_by_class_name('word')[:10]
        print(i*30,' 초째아직 추출중')
        i += 1
        time.sleep(30)

   
    # 파일 탭에서 추출 누르기, 전체화면 이어야 함
    browser.find_element_by_tag_name('button').click() # 탭
    browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div[2]/div[1]/div[1]/div[7]').click()
    browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div[2]/div[1]/div[1]/ul/li[2]/div/button').click()   
    browser.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[2]/div[2]/div[2]/label/span[1]').click()
    browser.find_element_by_class_name('blue-button').click() 
    print('추출완료')
    time.sleep(10)
    text_file = save_text(video)
    with open(save_file, 'a') as f:
         f.write('{},{},{}\n'.format(row['links'], row['videos'], text_file))
    print("저장완료")
    print('-'*50)    

In [ ]:
# ##중간에 에러났을 때 이어서 실행할 코드
# saved = idx

# save_file = os.path.join('.','data','links_videos_texts.txt')
# with open(save_file, 'a') as f:
#     f.write('links,videos,texts\n')


# # 브라우저 실행 
# browser = webdriver.Chrome("C:/Users/eunsun/tobigs/chromedriver.exe") #############본인경로에 맞게 변경
# browser.get('https://vrew.voyagerx.com/ko/try/')
# wait = WebDriverWait(browser, 4)
# element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'close')))
# browser.find_element_by_class_name('close').click()
# time.sleep(2)
# print('-'*50)

# # 크롤링 루프
# for i, row in df.iterrows():
#     if i<saved:
#         continue
#     video = row['videos'][2:]
#     print(video,'추출')
    
#     # 기본 변수 생성
#     path = os.getcwd()+'\\'+video ################변경
#     check = browser.find_elements_by_class_name('word')[:10]
#     new_check = browser.find_elements_by_class_name('word')[:10]
#     # 파일 탭으로 이동
#     browser.find_element_by_tag_name('button').click()
    
#     # 업로드 클릭
#     wait = WebDriverWait(browser, 5)
#     wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]')))
#     browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]').click()

#     # 업로드 실행
#     time.sleep(1)
#     app = pywinauto.application.Application().connect(title_re='열기')
#     app.열기.Edit.SetText(path)
#     time.sleep(1)
#     app.열기.Button.click()
#     try :
#         app.열기.Button.click()
#     except:
#         pass

#     # 변환 확인
#     wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'blue-button')))
#     browser.find_element_by_class_name('blue-button').click()
#     new_check = browser.find_elements_by_class_name('word')[:10]

#     i=0
#     while check == new_check:
#         new_check = browser.find_elements_by_class_name('word')[:10]
#         print(i*30,' 초째아직 추출중')
#         i += 1
#         time.sleep(30)

   
#     # 파일 탭에서 추출 누르기, 전체화면 이어야 함
#     browser.find_element_by_tag_name('button').click() # 탭
#     browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div[2]/div[1]/div[1]/div[7]').click()
#     browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div[2]/div[1]/div[1]/ul/li[2]/div/button').click()   
#     browser.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[2]/div[2]/div[2]/label/span[1]').click()
#     browser.find_element_by_class_name('blue-button').click() 
#     print('추출완료')
#     time.sleep(10)
#     text_file = save_text(video)
#     with open(save_file, 'a') as f:
#          f.write('{},{},{}\n'.format(row['links'], row['videos'], text_file))
#     print("저장완료")
#     print('-'*50)    

# video_to_audio.py

In [58]:
#!/usr/bin/env python
# coding: utf-8

# ## video audio 저장

import os
import pandas as pd
from moviepy.editor import *

def make_dir(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

def change_dir(from_dir, before='video', after='audio'):
    if after == 'audio':
        type_ = '.wav'
    elif after == 'text':
        type_ = '.txt'
    
    to_dir = from_dir.split('/')
    to_dir[-2] = after
    to_dir[-1] = to_dir[-1].split('.')[0] + type_
    return os.path.join(*to_dir)

def video_to_audio(video_dir):
    audio_dir = change_dir(video_dir, before='video', after='audio')
    print(video_dir)
    audioclip = VideoFileClip(video_dir).audio
    audioclip.write_audiofile(audio_dir)
    return audio_dir

def clip_audio(audio_dir):
    total = pd.DataFrame(columns=['audio','text', 'length'])
    
    text_dir = change_dir(audio_dir, before='audio_dir', after='text')
    
    df = pd.read_csv(text_dir, header=None, sep='\t')
    df.columns = ['time', 'text']
    
    for i in range(len(df)):
        full_audio = AudioFileClip(audio_dir)
        full_time = int(full_audio.end)
        
        tmp_dir = audio_dir[:-4] + '_{}.wav'.format(i)
        start_time = pd.Timedelta(df['time'][i]).total_seconds()
        if i != (len(df)-1):
            end_time = pd.Timedelta(df['time'][i+1]).total_seconds()
        else:
            end_time = full_time

        tmp_audio = full_audio.subclip(start_time, end_time)
        tmp_audio.write_audiofile(tmp_dir)
        
        total = total.append({'audio':tmp_dir, 
                              'text':df['text'][i], 
                              'length':(end_time-start_time)}, ignore_index=True)
    
    return total

# def labeling(df):
#     labels = []
#     for i, row in df.iterrows():
#         print('Text:', row['text'])
#         l = int(input('Label: '))
#         labels.append(l)
#     labels = pd.DataFrame({'labels':labels})
#     return df.join(labels)

In [59]:
videos = []
for root, dirs, files in os.walk('./data/video2/'):
    for fname in files:
        if fname.startswith('.DS'):
            continue
        videos.append(os.path.join(root, fname))


In [60]:
len(videos)

173

In [61]:
## if __name__ == "__main__"
audio_dir = os.path.join('.','data','audio')
make_dir(audio_dir)

df = pd.read_csv(os.path.join('.', 'data', 'links_videos_texts.txt'))
result = pd.DataFrame(columns=['audio', 'text', 'length'])#, 'label'])

for video in videos:
    video_dir = video
    audio_dir = video_to_audio(video_dir)
    try:
        clip_df = clip_audio(audio_dir)
    except IndexError:
        continue
    result = result.append(clip_df, ignore_index=True)

./data/video2/03Eu01fzY9I.mp4


chunk:   2%|▏         | 310/14984 [00:00<00:04, 3094.74it/s, now=None]

MoviePy - Writing audio in ./data/audio/03Eu01fzY9I.wav


MoviePy - Done.
MoviePy - Writing audio in ./data/audio/03Eu01fzY9I_0.wav


MoviePy - Done.
MoviePy - Writing audio in ./data/audio/03Eu01fzY9I_1.wav
MoviePy - Done.


...

MoviePy - Done.
MoviePy - Writing audio in ./data/audio/2zQBgMFbDaE_248.wav
MoviePy - Done.


chunk:   0%|          | 0/4347 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in ./data/audio/2zQBgMFbDaE_249.wav
./data/video2/iILrJ2286sc.mp4
MoviePy - Writing audio in ./data/audio/iILrJ2286sc.wav


MoviePy - Done.


FileNotFoundError: [Errno 2] File b'./data/text/iILrJ2286sc.txt' does not exist: b'./data/text/iILrJ2286sc.txt'

In [86]:
for video in videos[videos.index(video):]:
    video_dir = video
    audio_dir = video_to_audio(video_dir)
    try:
        clip_df = clip_audio(audio_dir)
    except IndexError:
        continue
    result = result.append(clip_df, ignore_index=True)

chunk:   0%|          | 0/4347 [00:00<?, ?it/s, now=None]

./data/video2/iILrJ2286sc.mp4
MoviePy - Writing audio in ./data/audio/iILrJ2286sc.wav


MoviePy - Done.
MoviePy - Writing audio in ./data/audio/iILrJ2286sc_0.wav
MoviePy - Done.


chunk:   0%|          | 0/45 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in ./data/audio/iILrJ2286sc_1.wav
MoviePy - Done.
MoviePy - Writing audio in ./data/audio/iILrJ2286sc_2.wav


...

MoviePy - Done.
MoviePy - Writing audio in ./data/audio/kjdRcQvlgUI_10.wav


chunk:   0%|          | 0/750 [00:00<?, ?it/s, now=None]            

MoviePy - Done.
MoviePy - Writing audio in ./data/audio/kjdRcQvlgUI_11.wav
MoviePy - Done.
MoviePy - Writing audio in ./data/audio/kjdRcQvlgUI_12.wav


MoviePy - Done.
MoviePy - Writing audio in ./data/audio/kjdRcQvlgUI_13.wav
MoviePy - Done.


MoviePy - Writing audio in ./data/audio/kjdRcQvlgUI_14.wav
MoviePy - Done.


In [ ]:
# ##중간에 멈췄을때 이어서 실행할 코드
# saved = _
# audio_dir = os.path.join('.','data','audio')
# make_dir(audio_dir)

# df = pd.read_csv(os.path.join('.', 'data', 'links_videos.txt'))
# result = pd.DataFrame(columns=['audio', 'text', 'length'])#, 'label'])

# for _, row in df.iterrows():
#     if _<saved:
#         continue
#     video_dir = row['videos']
#     audio_dir = video_to_audio(video_dir)
#     clip_df = clip_audio(audio_dir)
#     result = result.append(clip_df, ignore_index=True)
# #     labeling_df = labeling(clip_df)
# #     result = result.append(labeling_df, ignore_index=True)

In [87]:
result['label'] = ""

In [88]:
result.to_csv("audios_texts_length_labels.csv",encoding='ms949', index=False)

In [104]:
a = pd.read_csv("audios_texts_length_labels.csv",encoding='ms949')

In [105]:
import numpy

In [106]:
numpy.isnan(a['label'][0])

True

In [113]:
for i, row in a.iterrows():
    print(row['label'])
    if numpy.isnan(row['label']):
        a['label'][i]= 10
    print(row['label'])
    break

nan
nan


In [114]:
a

,audio,label,length,text
0,./data/audio/03Eu01fzY9I_0.wav,10.0,7.0,안녕하세요 오늘도 위에 여러분들의 응원에 힘입어서
1,./data/audio/03Eu01fzY9I_1.wav,NaN,8.0,단계를 먹방이 콘텐츠로 여러분들께 더 재밌고
2,./data/audio/03Eu01fzY9I_2.wav,NaN,5.0,더 신나는 약간 그 상상하지 못했던 단계별 먹고
3,./data/audio/03Eu01fzY9I_3.wav,NaN,8.0,오늘은 어떤 자격도 안 했거든
4,./data/audio/03Eu01fzY9I_4.wav,NaN,35.0,번지르르한 데 얼마나 아프면 첫 번째 계란 이게 뭐야 652
...,...,...,...,...
7500,./data/audio/kjdRcQvlgUI_10.wav,NaN,28.0,여동생 남자라고요 나랑 하면 되는데 왜 통화했는데
7501,./data/audio/kjdRcQvlgUI_11.wav,NaN,2.0,혼자야 짜증 난다
7502,./data/audio/kjdRcQvlgUI_12.wav,NaN,34.0,남자 만나 머리를 쓰다듬는 거야 뭐야 진짜 좋은 거야
7503,./data/audio/kjdRcQvlgUI_13.wav,NaN,15.0,안 보인다고 진짜 너무 솔직하게 써도 들어봤는데
